# 1. 결정 트리 훈련

1-a . make_moons (n_sample=1000, noise=0.4)를 사용해 데이터셋을 생성합니다.

In [56]:
from sklearn.datasets import make_moons

X,y = make_moons(n_samples=1000, noise=0.4, random_state=42)

1-b. 이를 train_test_split( )을 사용해 훈련 세트와 테스트 세트로 나눕니다.(random_state=42, train:test = 8:2 비율로)


In [57]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

1-c. DecisionTreeClassifier의 최적의 매개변수를 찾기 위해 교차 검증과 함께 그리드 탐색을 수행합니다. (GridSearchCV)

힌트 : 여러가지 max_leaf_nodes 값을 시도해보세요! (교재 pg. 113, 207 참고)

In [58]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# 예시 파라미터: params = {'max_leaf_nodes' : list(range(2,100)),'min_samples_split' : [2,3,4]}

dt_clf = DecisionTreeClassifier()

params = {
    'max_leaf_nodes' : list(range(2,100)),
    'min_samples_split' : [2,3,4]
}

grid_search_cv = GridSearchCV(dt_clf, param_grid=params, scoring='accuracy', cv=5, verbose=1)
grid_search_cv.fit(X_train, y_train)

Fitting 5 folds for each of 294 candidates, totalling 1470 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15, 16, 17, 18, 19, 20, 21,
                                            22, 23, 24, 25, 26, 27, 28, 29, 30,
                                            31, ...],
                         'min_samples_split': [2, 3, 4]},
             scoring='accuracy', verbose=1)

In [59]:
# 최적의 매개변수 확인
grid_search_cv.best_estimator_

DecisionTreeClassifier(max_leaf_nodes=6)

1-d. 찾은 매개변수를 사용해 전체 훈련 세트에 대해 모델을 훈련시키고 테스트 세트에서 성능(정확도)을 측정합니다. (대략 85~87% 정도 나옵니다.)

In [60]:
# GridSearchCV는 최적의 모델로 다시 훈련시키기 때문에 별도의 훈련이 필요없습니다.
from sklearn.metrics import accuracy_score
y_pred = grid_search_cv.best_estimator_.predict(X_test)
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test , y_pred)))

예측 정확도: 0.8550


# 2. 랜덤포레스트

In [61]:
# 데이터 다운로드 링크로 데이터를 코랩에 불러옵니다.

!wget 'https://bit.ly/3i4n1QB'

import zipfile
with zipfile.ZipFile('3i4n1QB', 'r') as existing_zip:
    existing_zip.extractall('data')

--2023-10-09 10:06:51--  https://bit.ly/3i4n1QB
Resolving bit.ly (bit.ly)... 67.199.248.10, 67.199.248.11
Connecting to bit.ly (bit.ly)|67.199.248.10|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw [following]
--2023-10-09 10:06:51--  https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw
Resolving drive.google.com (drive.google.com)... 173.194.215.139, 173.194.215.102, 173.194.215.138, ...
Connecting to drive.google.com (drive.google.com)|173.194.215.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-10-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/g9kc87udsvs7ipci9b6760bj8lupa5fm/1696845975000/17946651057176172524/*/1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw?e=download&uuid=26c12f94-9509-4058-9ce9-d0633729d410 [following]
--2023-10-09 10:06:51--  https://doc-10-10-doc

In [62]:
# 라이브러리 불러오기

import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

In [63]:
train = pd.read_csv('data/train.csv')

# Scaling
scaler = MinMaxScaler()
scaler.fit(train[['fixed acidity']])
train['Scaled fixed acidity'] = scaler.transform(train[['fixed acidity']])

# Encoding
encoder = OneHotEncoder()
encoder.fit(train[['type']])
onehot = encoder.transform(train[['type']])
onehot = onehot.toarray()
onehot = pd.DataFrame(onehot)
onehot.columns =encoder.get_feature_names_out()
train = pd.concat([train, onehot], axis = 1)
train = train.drop(columns = ['type'])
train.head()

,index,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,Scaled fixed acidity,type_red,type_white
0,0,5,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,0.148760,0.0,1.0
1,1,5,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,0.413223,1.0,0.0
2,2,5,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,0.338843,0.0,1.0
3,3,6,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,0.264463,0.0,1.0
4,4,6,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,0.330579,0.0,1.0


2-a. 랜덤포레스트 분류 모형을 불러옵니다.

In [64]:
from sklearn.ensemble import RandomForestClassifier

2-b. 랜덤포레스트 분류 모형을 "random_classifier"라는 변수에 저장합니다.

In [65]:
random_classifier = RandomForestClassifier()

2-c. "X"라는 변수에 train의 "quality" 피쳐를 제거하고 저장합니다.

In [66]:
X = train.iloc[:,2:]
X

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,Scaled fixed acidity,type_red,type_white
0,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,0.148760,0.0,1.0
1,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,0.413223,1.0,0.0
2,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,0.338843,0.0,1.0
3,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,0.264463,0.0,1.0
4,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,0.330579,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5492,7.7,0.150,0.29,1.3,0.029,10.0,64.0,0.99320,3.35,0.39,10.1,0.322314,0.0,1.0
5493,6.3,0.180,0.36,1.2,0.034,26.0,111.0,0.99074,3.16,0.51,11.0,0.206612,0.0,1.0
5494,7.8,0.150,0.34,1.1,0.035,31.0,93.0,0.99096,3.07,0.72,11.3,0.330579,0.0,1.0
5495,6.6,0.410,0.31,1.6,0.042,18.0,101.0,0.99195,3.13,0.41,10.5,0.231405,0.0,1.0


2-d. "y"라는 변수에 정답인 train의 "quality" 피쳐를 저장합니다.

In [67]:
y = train.iloc[:,1]
y

0       5
1       5
2       5
3       6
4       6
       ..
5492    5
5493    6
5494    7
5495    5
5496    6
Name: quality, Length: 5497, dtype: int64

2-e. "random_classifier"를 X와 y를 이용해 학습시켜보세요.

In [68]:
random_classifier.fit(X,y)

RandomForestClassifier()